# IMPORT GLOBALES

In [1]:
import pandas as pd
import numpy as np
import  requests
import io

# VARIABLES GLOBALES

In [2]:
API_KEY="ZzBI5ufE20w8HjCTloldVGL3KmN5aidAM5QsdmL9"
DATASET_VELOCIDAD_BAJADA_PROVINCIA="VELOC-PROME-DE-BAJAD-DE"
DATASET_ACCESOS_VELOCIDAD_PROVINCIA="ACCES-A-INTER-FIJO-23248"

#LECTURA DE ARCHIVOS

In [3]:
def get_api_df(dataset):
  url = f"https://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/{dataset}/data.csv/?auth_key={API_KEY}"

  return pd.read_csv(url, encoding="utf-8")

#### Número total de accesos al servicio de Internet fijo por banda ancha y banda angosta (trimestral)

In [4]:
df_velocidad_bajada = get_api_df(DATASET_VELOCIDAD_BAJADA_PROVINCIA)
df_accesos_velocidad = get_api_df(DATASET_ACCESOS_VELOCIDAD_PROVINCIA)

In [5]:
df_velocidad_bajada

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2022,4,Buenos Aires,111.000
1,2022,4,Capital Federal,18.286
2,2022,4,Catamarca,8.333
3,2022,4,Chaco,10.402
4,2022,4,Chubut,1.592
...,...,...,...,...
859,2014,1,Santa Cruz,332.000
860,2014,1,Santa Fe,315.000
861,2014,1,Santiago Del Estero,295.000
862,2014,1,Tierra Del Fuego,345.000


In [6]:
df_accesos_velocidad

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2022,4,Buenos Aires,28.934,23.347,259.866,281.550,244.555,98.670,3.760.109,97.634,4.794.665
1,2022,4,Capital Federal,516.000,5.096,31.320,64.653,36.336,8.116,1.299.117,106.772,1.551.926
2,2022,4,Catamarca,71.000,315.000,2.716,5.028,4.638,3.122,52.739,2.280,70.909
3,2022,4,Chaco,461.000,771.000,16.067,17.846,6.704,3.039,94.866,3.909,143.663
4,2022,4,Chubut,109.000,1.614,46.457,31.007,34.728,15.770,19.048,20.026,168.759
...,...,...,...,...,...,...,...,...,...,...,...,...
859,2014,1,Santa Cruz,161.000,1.625,24.972,1.000,1.000,0.000,0,0.000,26.760
860,2014,1,Santa Fe,8.456,124.468,345.225,20.328,6.845,23.000,668,0.000,506.013
861,2014,1,Santiago Del Estero,1.234,10.531,22.817,2.422,109.000,0.000,0,0.000,37.113
862,2014,1,Tierra Del Fuego,12.000,607.000,30.902,6.000,0.000,0.000,0,0.000,31.527


In [7]:
(df_accesos_velocidad["OTROS"]==0.000).sum()/df_accesos_velocidad["OTROS"].count()

0.5268065268065268

In [8]:
df_velocidad_bajada.rename(columns = {'Mbps (Media de bajada)':'Mbps'}, inplace = True)
df_velocidad_bajada

,Año,Trimestre,Provincia,Mbps
0,2022,4,Buenos Aires,111.000
1,2022,4,Capital Federal,18.286
2,2022,4,Catamarca,8.333
3,2022,4,Chaco,10.402
4,2022,4,Chubut,1.592
...,...,...,...,...
859,2014,1,Santa Cruz,332.000
860,2014,1,Santa Fe,315.000
861,2014,1,Santiago Del Estero,295.000
862,2014,1,Tierra Del Fuego,345.000


In [9]:
df_accesos_velocidad = df_accesos_velocidad.drop(['OTROS','Total'], axis=1)
df_accesos_velocidad.rename(columns = {'HASTA 512 kbps':'512kbps','+ 512 Kbps - 1 Mbps':'512kbps-1mbps','+ 1 Mbps - 6 Mbps':'1mbps-6mbps','+ 6 Mbps - 10 Mbps':'6mbps-10mbps','+ 10 Mbps - 20 Mbps':'10mbps-20mbps','+ 20 Mbps - 30 Mbps':'20mbps-30mbps','+ 30 Mbps':'30mbps'}, inplace = True)
df_accesos_velocidad

,Año,Trimestre,Provincia,512kbps,512kbps-1mbps,1mbps-6mbps,6mbps-10mbps,10mbps-20mbps,20mbps-30mbps,30mbps
0,2022,4,Buenos Aires,28.934,23.347,259.866,281.550,244.555,98.670,3.760.109
1,2022,4,Capital Federal,516.000,5.096,31.320,64.653,36.336,8.116,1.299.117
2,2022,4,Catamarca,71.000,315.000,2.716,5.028,4.638,3.122,52.739
3,2022,4,Chaco,461.000,771.000,16.067,17.846,6.704,3.039,94.866
4,2022,4,Chubut,109.000,1.614,46.457,31.007,34.728,15.770,19.048
...,...,...,...,...,...,...,...,...,...,...
859,2014,1,Santa Cruz,161.000,1.625,24.972,1.000,1.000,0.000,0
860,2014,1,Santa Fe,8.456,124.468,345.225,20.328,6.845,23.000,668
861,2014,1,Santiago Del Estero,1.234,10.531,22.817,2.422,109.000,0.000,0
862,2014,1,Tierra Del Fuego,12.000,607.000,30.902,6.000,0.000,0.000,0


In [10]:
df_accesos_velocidad = pd.merge(df_accesos_velocidad, df_velocidad_bajada[["Año","Trimestre","Mbps","Provincia"]], on=["Año","Trimestre","Provincia"], how="left")
df_accesos_velocidad

,Año,Trimestre,Provincia,512kbps,512kbps-1mbps,1mbps-6mbps,6mbps-10mbps,10mbps-20mbps,20mbps-30mbps,30mbps,Mbps
0,2022,4,Buenos Aires,28.934,23.347,259.866,281.550,244.555,98.670,3.760.109,111.000
1,2022,4,Capital Federal,516.000,5.096,31.320,64.653,36.336,8.116,1.299.117,18.286
2,2022,4,Catamarca,71.000,315.000,2.716,5.028,4.638,3.122,52.739,8.333
3,2022,4,Chaco,461.000,771.000,16.067,17.846,6.704,3.039,94.866,10.402
4,2022,4,Chubut,109.000,1.614,46.457,31.007,34.728,15.770,19.048,1.592
...,...,...,...,...,...,...,...,...,...,...,...
859,2014,1,Santa Cruz,161.000,1.625,24.972,1.000,1.000,0.000,0,332.000
860,2014,1,Santa Fe,8.456,124.468,345.225,20.328,6.845,23.000,668,315.000
861,2014,1,Santiago Del Estero,1.234,10.531,22.817,2.422,109.000,0.000,0,295.000
862,2014,1,Tierra Del Fuego,12.000,607.000,30.902,6.000,0.000,0.000,0,345.000


# ANALISIS

## ANALISIS GENERAL

In [ ]:
df_accesos_velocidad.head()

,Año,Trimestre,Provincia,512kbps,512kbps-1mbps,1mbps-6mbps,6mbps-10mbps,10mbps-20mbps,20mbps-30mbps,30mbps,Mbps
0,2022,4,Buenos Aires,28.934,23.347,259.866,281.550,244.555,98.670,3.760.109,111.000
1,2022,4,Capital Federal,516.000,5.096,31.320,64.653,36.336,8.116,1.299.117,18.286
2,2022,4,Catamarca,71.000,315.000,2.716,5.028,4.638,3.122,52.739,8.333
3,2022,4,Chaco,461.000,771.000,16.067,17.846,6.704,3.039,94.866,10.402
4,2022,4,Chubut,109.000,1.614,46.457,31.007,34.728,15.770,19.048,1.592


In [ ]:
df_accesos_velocidad.columns

Index(['Año', 'Trimestre', 'Provincia', '512kbps', '512kbps-1mbps',
       '1mbps-6mbps', '6mbps-10mbps', '10mbps-20mbps', '20mbps-30mbps',
       '30mbps', 'Mbps'],
      dtype='object')

In [ ]:
df_accesos_velocidad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 864 entries, 0 to 863
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Año            864 non-null    int64  
 1   Trimestre      864 non-null    int64  
 2   Provincia      864 non-null    object 
 3   512kbps        864 non-null    float64
 4   512kbps-1mbps  864 non-null    float64
 5   1mbps-6mbps    864 non-null    object 
 6   6mbps-10mbps   864 non-null    float64
 7   10mbps-20mbps  864 non-null    float64
 8   20mbps-30mbps  864 non-null    float64
 9   30mbps         864 non-null    object 
 10  Mbps           864 non-null    float64
dtypes: float64(6), int64(2), object(3)
memory usage: 81.0+ KB


In [ ]:
df_accesos_velocidad.describe()

,Año,Trimestre,512kbps,512kbps-1mbps,6mbps-10mbps,10mbps-20mbps,20mbps-30mbps,Mbps
count,864.000000,864.000000,864.000000,864.000000,864.000000,864.000000,864.000000,864.000000
mean,2018.000000,2.500000,126.069829,104.202722,70.717505,78.652155,99.993059,302.410067
std,2.583484,1.118682,186.417712,211.789353,138.831868,168.570068,207.349280,287.314571
min,2014.000000,1.000000,1.007000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2016.000000,1.750000,11.000000,3.310750,5.410250,5.000000,2.091000,2.808500
50%,2018.000000,2.500000,51.000000,8.762000,20.208000,15.159000,11.042500,344.000000
75%,2020.000000,3.250000,133.250000,85.525500,61.934500,51.959500,64.286250,520.250000
max,2022.000000,4.000000,998.000000,999.000000,917.000000,978.000000,997.000000,988.000000


## TRATAMIENTO DE DATOS NULOS

In [ ]:
df_accesos_velocidad.isnull().sum()

Año              0
Trimestre        0
Provincia        0
512kbps          0
512kbps-1mbps    0
1mbps-6mbps      0
6mbps-10mbps     0
10mbps-20mbps    0
20mbps-30mbps    0
30mbps           0
Mbps             0
dtype: int64

# EXPORTAR DATOS

In [ ]:
df_accesos_velocidad.to_csv('data/velocidad.csv', index=False)